<img src="http://www.cs.wm.edu/~rml/images/wm_horizontal_single_line_full_color.png">

<h1 style="text-align:center;">CSCI 141, Fall 2020</h1>
<h1 style="text-align:center;">Creating maps with folium</h1>

# Contents:
* [Building maps](#maps)
* [A simple map](#simple_map)
* [An earthquake map](#earthquake_map)
* [The project](#project)

One of the great strengths of Python is its enormous software ecosystem which makes it possible to do interesting things without having to write lots of code. 

This example uses the following Python module:
* [folium](https://github.com/python-visualization/folium/blob/master/README.rst), a map-making library.

First we need to install the folium module.  The following cell only needs to be executed once.

This may take a few seconds&hellip;

In [ ]:
# This cell need only be executed once.  It will install the folium package, 
# which is not part of Anaconda.

# pip is the standard utility for installing/updating/... Python modules.

!pip install folium

# A simple map <a id="simple_map"/>

Let's draw a map that opens on the center of the universe.  The <code>location</code> argument indicates the center of the map.

In [ ]:
import folium
wm = folium.Map(location=[37.270833, -76.709167], zoom_start=17)
wm

This illustrates the power of the Python software ecosystem &ndash; with three lines of code we're done.

You can zoom in and out, and a left click allows you to drag the map to another location.

Now we'll use a little Jupyter/HTML magic to add a title.  First we save the map to an [HTML](https://en.wikipedia.org/wiki/HTML) file:

In [ ]:
wm.save('wm.html')

The <code>%%HTML</code> in the next cell indicates that this is to be interpreted as HTML rather than Python or markup language.  The HTML <code>&lt;iframe&gt;</code> tag allows us to embed on webpage in another.

In [ ]:
%%HTML
<h1>Our first map</h1> 
<iframe width="100%" height="500" src="wm.html"></iframe>

We can also annotate the map with pins and specify text that appears when the pin is clicked on:

In [ ]:
wm = folium.Map(location=[37.270833, -76.709167], zoom_start=17)

lat =   37 + 16/60 + 14.80/3600  # Convert minutes and seconds to decimal latitude
lon = -(76 + 42/60 + 32.23/3600) # and longitude.
folium.Marker(location=[lat, lon], popup='Wren Building',
              icon=folium.Icon(icon='university', prefix='fa')).add_to(wm)  

lat =   37 + 16/60 + 12.50/3600
lon = -(76 + 42/60 + 42.50/3600)
folium.Marker(location=[lat, lon], popup='The dumpy home of CS', 
              icon=folium.Icon(icon='desktop', prefix='fa')).add_to(wm)

lat =   37 + 16/60 + 14.80/3600  # Convert minutes and seconds to decimal latitude
lon = -(76 + 42/60 + 39/3600) # and longitude.
folium.Marker(location=[lat, lon], popup='Sunken Garden', tooltip='Sunken Garden').add_to(wm) 

wm

# Building an earthquake map <a id="earthquake_map"/>

The next example downloads data from the United States Geological Survey (USGS) to map the location and intensity of all earthquakes recorded world-wide in the past 24 hours.

This example is based on
* [https://blog.dominodatalab.com/lesser-known-ways-of-using-notebooks](https://blog.dominodatalab.com/lesser-known-ways-of-using-notebooks)

In [ ]:
import pandas as pd  # Pandas data analysis package.
from matplotlib.colors import Normalize, rgb2hex  # Matplotlib plotting package.
import matplotlib.cm as cm  # A color manager.

First we read the data file from the USGS website.  The USGS updates this file every 5 minutes.

In [ ]:
eq = pd.read_csv('http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.csv')

This reads the file from the web as a Pandas **dataframe**.  Let's take a look at the first few entries.

In [ ]:
# Note the column headers. 
eq.head()

We can refer to the columns using the header as a key:

In [ ]:
print(eq['mag'])
print(eq['place'])

We can restrict attention to rows where some condition is satisfied.

For example, it is possible to have [negative magnitude earthquakes](https://www.usgs.gov/faqs/how-can-earthquake-have-a-negative-magnitude?qt-news_science_products=0#qt-news_science_products).  We can extract any of those as follows:

In [ ]:
eq[eq['mag'] < 0]

This should be read as "show the rows of eq where the magnitude is negative".

We can iterate over the rows as follows.  Note that the items we are iterating over are a tuple.  The first element is the row number, and the second is the actual row data.  If you see any Nans this is because Pandas uses the not-a-number float value if a field is empty in the CSV.

In [ ]:
for quake in eq.iterrows():
    print(quake)

Let's turn this data into a map.

We create circles on the map for each earthquake.  Each circle is centered on the earthquake's epicenter, and the size and color of the circle reflect the earthquake's magnitude.  We set things up so that clicking on a circle reveals the location and magnitude.

The resulting map is written to the file <code>earthquake.html</code>.

In [ ]:
from folium.plugins import MarkerCluster

norm = Normalize(eq['mag'].min(), eq['mag'].max())  # Build a function to normalize the data.

eq_map = folium.Map(location=[48, -102], zoom_start=1)  # Zoom all the way out to see Earth.

# Iterate over the earthquakes.
print('generating map...', end='')
for qk in eq.iterrows():
    color = rgb2hex(cm.OrRd(norm(float(qk[1]['mag']))))
    folium.CircleMarker([qk[1]['latitude'], qk[1]['longitude']], 
                        popup=f"{qk[1]['place']} magnitude: {qk[1]['mag']}",
                        radius=2*float(qk[1]['mag']),
                        color=color,
                        fill_color=color).add_to(eq_map)
print('done!')
eq_map.save(outfile='earthquake.html')
eq_map

We can also cluster markers together, which is useful if there are a large number of them.  Clicking on a cluster or zooming in splits the cluster into smaller units; zooming out causes clusters to agglomerate.

In [ ]:
from folium.plugins import MarkerCluster

norm = Normalize(eq['mag'].min(), eq['mag'].max())

eq_map = folium.Map(location=[48, -102], zoom_start=1)
eq_cluster = MarkerCluster().add_to(eq_map)

# Iterate over the earthquakes.
print('generating map...', end='')
for qk in eq.iterrows():
    color = rgb2hex(cm.OrRd(norm(float(qk[1]['mag']))))
    folium.CircleMarker([qk[1]['latitude'], qk[1]['longitude']], 
                        popup=f"{qk[1]['place']} magnitude: {qk[1]['mag']}",
                        radius=2*float(qk[1]['mag']),
                        color=color,
                        fill_color=color).add_to(eq_cluster)
print('done!')
# Save the map.    
# eq_map.save(outfile='earthquake.html')
eq_map

Next, let's combine these two constructions.  This makes the code easier to maintain and keep consistent.

In [ ]:
norm = Normalize(eq['mag'].min(), eq['mag'].max())

eq_map = []
for i in range(2):
    eq_map.append(folium.Map(location=[48, -102], zoom_start=1))
eq_cluster = MarkerCluster().add_to(eq_map[1])

targets = [eq_map[0], eq_cluster]  # Build a map without and with marker clusters.

# Iterate over the earthquakes.
print('generating map...', end='')
for target in targets:
    for qk in eq.iterrows():
        color = rgb2hex(cm.OrRd(norm(float(qk[1]['mag']))))
        folium.CircleMarker([qk[1]['latitude'], qk[1]['longitude']], 
                        popup=f"{qk[1]['place']} magnitude: {qk[1]['mag']}",
                        radius=2*float(qk[1]['mag']),
                        color=color,
                        fill_color=color).add_to(target)
print('done!')

First, the map without marker clusters:

In [ ]:
eq_map[0]

Second, the map with marker clusters:

In [ ]:
eq_map[1]

# The project (with solution) <a id="project"/>

In this project you will create a Jupyter notebook named <code>awois.ipynb</code> that maps the data in [NOAA's Automated Wreck and Obstruction Information System (AWOIS)](https://www.nauticalcharts.noaa.gov/data/wrecks-and-obstructions.html).  AWOIS
<blockquote>
contains information on over 10,000 submerged wrecks and obstructions in the coastal waters of the United States. Information includes latitude and longitude of each feature along with brief historic and descriptive details.
</blockquote>

## The data

Your notebook should use the Pandas [<code>read_csv()</code> function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) to read the data from [http://www.cs.wm.edu/~rml/teaching/csci141/data/AWOIS_Wrecks.csv](http://www.cs.wm.edu/~rml/teaching/csci141/data/AWOIS_Wrecks.csv) into a Pandas dataframe.

You will need to pass the argument <code>encoding='cp1252'</code> to <code>read_csv()</code>.  This CSV file contains non-ascii characters (e.g., &#176;) and you'll encounter an error if you do not specify this encoding, which is [an old encoding used by Microsoft for Western European languages](https://en.wikipedia.org/wiki/Windows-1252).

In [ ]:
import folium
import pandas as pd

wrecks = pd.read_csv('http://www.cs.wm.edu/~rml/teaching/csci141/data/AWOIS_Wrecks.csv', encoding='cp1252')  # Windows 1252 encoding.

print(wrecks.columns)

The names of the columns in this file are 
<pre>
'RECRD', 'VESSLTERMS', 'FEATURE_TYPE', 'LATDEC', 'LONDEC', 'GP_QUALITY',
'DEPTH', 'SOUNDING_TYPE', 'YEARSUNK', 'HISTORY'
</pre>
These are described in the [AWOIS User's Guide](http://www.cs.wm.edu/~rml/teaching/csci141/data/awois-users-guide-2013.pdf).

You should spend some time looking at the dataframe so you understand how it is organized.

## Map 1: without clusters

Your map should map all of the wrecks in the AWOIS database.
* Center the map on 37N, -76W, which is the mouth of Hampton Roads.
* Use <code>folium.Marker</code> to indicate the wrecks.
* Use the latitude and longitude of a wreck to map it.
* The <code>popup</code> argument for each marker should be the wreck's <code>VESSLTERMS</code> attribute followed by the wreck's <code>FEATURE_TYPE</code> attribute (you'll need to combine these in a single string).

Because there are thousands of markers on this map, it will likely take several seconds to display.

## Map 2: with clusters

This map is almost identical to the previous map, only with the markers organized using <code>folium.MarkerCluster</code>.

## Map 3: ships sunk by submarines

During the First and Second World Wars, hundreds of ships were sunk in the Gulf of Mexico and off the Atlantic coast of the US by German submarines.  The worst time near the US coast was the first half of 1942; [this list from Wikipedia shows how bad it was](https://en.wikipedia.org/wiki/Second_Happy_Time#Chronology_of_attacks).

This map will indicate ships sunk by submarines with a red marker with an anchor icon.  You can add an anchor icon by passing the argument
<pre>
icon=folium.Icon(color='red',prefix='fa',icon='anchor')
</pre>
to <code>folium.Marker</code>.  The prefix <code>'fa'</code> stands for [Font Awesome](https://fontawesome.com/icons?d=gallery&m=free).

### Finding the ships using regular expressions

Determining whether a ship was sunk by a submarine is a bit of a challenge.  There is a notation in the <code>HISTORY</code> field, but it varies.  Here are some examples:
<pre>
SUNK 10/6/42 BY SUBMARINE
SUNK 1942 BY SUBMARINE
SUNK 6/2/18 BY GERMAN SUBMARINE
</pre>
Moreover, we cannot find these simply by searching for <code>SUNK</code> because of strings like this:
<pre>
96 FT TUG, SUNK IN APPROX. 72 FT OF WATER
</pre>
nor can we find them by searching for <code>SUBMARINE</code> (or both words) because of strings like this:
<pre>
SUBMARINE, 740 GT; SUNK 6/2/43
</pre>

This is where we can use [**regular expressions**](https://xkcd.com/208/) (**regex** or **regexp** for short).  Regular expressions are a way of expressing a pattern to search for.  We will look at them in greater depth later, but here is how we can use them here.

A perusal of the file reveals all of the references to ships sunk by submarines have the form
<pre>
SUNK (date) BY (zero or one word) SUBMARINE
</pre>
We can capture this with the regex
<pre>
SUNK.+BY.+SUBMARINE
</pre>
In this context, <code>.+</code> means "one or more characters of any type".  We could be more precise in specifying the pattern, but this will suffice for our purposes.

Now we will use Python's [<code>re</code> module](https://docs.python.org/3/library/re.html).

In [ ]:
import re

# Construct the search engine for our pattern.
p = re.compile('SUNK.+ BY.+SUBMARINE')

# Here are some strings to search.
lines = ['(SOURCE UNK); POSITION ACCURACY 1 MILE; WD CLEAR TO 36 FT REPORTED'
         'NO.1596; TRAWLER; POSITION ACCURACY WITHIN 1 MILE.',
         'NO.436; POSITION ACCURACY WITHIN 1 MILE AT 35-13-30N, 75-11-42W;',
         'NO.1633; POSITION ACCURACY WITHIN 1 MILE',
         'NO.4623; CARGO, 7874 GT,SUNK 3/7/42 BY GERMAN SUBMARINE, POSITION ACCURACY 3-5',
         'NO.860; CARGO, 3779 GT,SUNK 1942 BY SUBMARINE; POSITION ACCURACY 1-3',
         'NO.4619; CARGO, 4834 GT,SUNK 4/16/42 BY SUBMARINE, POSITION ACCURACY 3-5',
         'NO.4624; CARGO, 1698 GT,SUNK 3/16/42 BY SUBMARINE; POSITION ACCURACY 3-5']

# Print the lines that contain the pattern.
for line in lines:
    if p.search(line):
        print(line)

# Print only the part of the line that matches the pattern.        
print(72*'-')        
for line in lines:
    s = p.search(line)
    if s:  # Evaluates to True if s is non-empty.
        print(s.group())

# Same as the preceding, only using an assignment expression (the walrus)
# which is new in Python 3.8.
print(72*'-')        
for line in lines:        
    if (s := p.search(line)):  # I am the walrus!  This is an assignment expression; 
        print(s.group())       # it assigns s a value and uses it in the if statement.

In your map, you should use the <code>group()</code> method to capture the pattern and add this information to your popup string.

**NB:** some rows are missing a <code>HISTORY</code> field so that are NaNs in the Pandas dataframe.  You will need to check that you are dealing with a string before searching for the pattern:

In [ ]:
# Here are some things to search (or not).
lines = ['(SOURCE UNK); POSITION ACCURACY 1 MILE; WD CLEAR TO 36 FT REPORTED'
         'NO.1596; TRAWLER; POSITION ACCURACY WITHIN 1 MILE.',
         'NO.436; POSITION ACCURACY WITHIN 1 MILE AT 35-13-30N, 75-11-42W;',
         'NO.1633; POSITION ACCURACY WITHIN 1 MILE',
          float('NaN'),
         'NO.4623; CARGO, 7874 GT,SUNK 3/7/42 BY GERMAN SUBMARINE, POSITION ACCURACY 3-5',
         'NO.860; CARGO, 3779 GT,SUNK 1942 BY SUBMARINE; POSITION ACCURACY 1-3',
         'NO.4619; CARGO, 4834 GT,SUNK 4/16/42 BY SUBMARINE, POSITION ACCURACY 3-5',
         'NO.4624; CARGO, 1698 GT,SUNK 3/16/42 BY SUBMARINE; POSITION ACCURACY 3-5']

# Print the lines that contain the pattern.
for line in lines:
    print(line)
    if p.search(line):  # Oops!
        print(line)

In [ ]:
# Here are some things to search (or not).
lines = ['(SOURCE UNK); POSITION ACCURACY 1 MILE; WD CLEAR TO 36 FT REPORTED'
         'NO.1596; TRAWLER; POSITION ACCURACY WITHIN 1 MILE.',
         'NO.436; POSITION ACCURACY WITHIN 1 MILE AT 35-13-30N, 75-11-42W;',
         'NO.1633; POSITION ACCURACY WITHIN 1 MILE',
          float('nan'),
         'NO.4623; CARGO, 7874 GT,SUNK 3/7/42 BY GERMAN SUBMARINE, POSITION ACCURACY 3-5',
         'NO.860; CARGO, 3779 GT,SUNK 1942 BY SUBMARINE; POSITION ACCURACY 1-3',
         'NO.4619; CARGO, 4834 GT,SUNK 4/16/42 BY SUBMARINE, POSITION ACCURACY 3-5',
         'NO.4624; CARGO, 1698 GT,SUNK 3/16/42 BY SUBMARINE; POSITION ACCURACY 3-5']

# Print the lines that contain the pattern.
for line in lines:
    if isinstance(line, str) and p.search(line):
        print(line)

### The marker cluster

Your map should have one marker cluster, one for the vessels **not** sunk by submarines.  The markers for those sunk by submarines should not be clustered.

## What you should submit

Upload your Jupyter notebook <code>awois.ipynb</code> to the Blackboard site.